In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH04-Models")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH04-Models


# 직렬화(Serialization)

In [3]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("{fruit}의 색상이 무엇입니까?")

In [4]:
print(f"ChatOpenAI: {ChatOpenAI.is_lc_serializable()}")

ChatOpenAI: True


In [5]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

print(f"ChatOpenAI: {llm.is_lc_serializable()}")

ChatOpenAI: True


In [6]:
chain = prompt | llm

chain.is_lc_serializable()

True

In [7]:
from langchain_core.load import dumpd, dumps

dumpd_chain = dumpd(chain)
dumpd_chain

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'schema', 'runnable', 'RunnableSequence'],
 'kwargs': {'first': {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'prompts', 'prompt', 'PromptTemplate'],
   'kwargs': {'input_variables': ['fruit'],
    'template': '{fruit}의 색상이 무엇입니까?',
    'template_format': 'f-string'},
   'name': 'PromptTemplate'},
  'last': {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'chat_models', 'openai', 'ChatOpenAI'],
   'kwargs': {'model_name': 'gpt-4o-mini',
    'temperature': 0.0,
    'openai_api_key': {'lc': 1, 'type': 'secret', 'id': ['OPENAI_API_KEY']}},
   'name': 'ChatOpenAI'}},
 'name': 'RunnableSequence'}

In [8]:
type(dumpd_chain)

dict

In [9]:
# dumps 함수를 사용하여 직렬화된 체인을 확인합니다.
dumps_chain = dumps(chain)
dumps_chain

'{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "runnable", "RunnableSequence"], "kwargs": {"first": {"lc": 1, "type": "constructor", "id": ["langchain", "prompts", "prompt", "PromptTemplate"], "kwargs": {"input_variables": ["fruit"], "template": "{fruit}\\uc758 \\uc0c9\\uc0c1\\uc774 \\ubb34\\uc5c7\\uc785\\ub2c8\\uae4c?", "template_format": "f-string"}, "name": "PromptTemplate"}, "last": {"lc": 1, "type": "constructor", "id": ["langchain", "chat_models", "openai", "ChatOpenAI"], "kwargs": {"model_name": "gpt-4o-mini", "temperature": 0.0, "openai_api_key": {"lc": 1, "type": "secret", "id": ["OPENAI_API_KEY"]}}, "name": "ChatOpenAI"}}, "name": "RunnableSequence"}'

In [10]:
type(dumps_chain)

str

# Pickle 파일

In [12]:
import pickle

with open("fruit_chain.pkl", "wb") as f:
    pickle.dump(dumpd_chain, f)

In [14]:
import json
with open("fruit_chain.json", "w") as fp:
    json.dump(dumpd_chain, fp)

# load: 저장한 모델 불러오기

In [15]:
import pickle

with open("fruit_chain.pkl", "rb") as f:
    loaded_chain = pickle.load(f)

In [16]:
from langchain_core.load import load

chain_from_file = load(loaded_chain)

print(chain_from_file.invoke({"fruit": "사과"}))

C:\Users\kakao\AppData\Local\Temp\ipykernel_10860\3667388523.py:3: LangChainBetaWarning: The function `load` is in beta. It is actively being worked on, so the API may change.
  chain_from_file = load(loaded_chain)


content='사과의 색상은 다양합니다. 일반적으로 빨간색, 초록색, 노란색 등 여러 가지 색상이 있으며, 품종에 따라 다르게 나타납니다. 예를 들어, 레드 딜리셔스는 주로 빨간색이고, 그라니 스미스는 초록색이며, 골든 딜리셔스는 노란색입니다. 또한, 일부 사과는 두 가지 이상의 색상이 섞여 있는 경우도 있습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 103, 'prompt_tokens': 16, 'total_tokens': 119, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_86d0290411', 'id': 'chatcmpl-BFvolWGsL4xaLlUl9iKHrDNwF5qZI', 'finish_reason': 'stop', 'logprobs': None} id='run-91aa7813-543f-4bdd-80b0-52fb6c4df288-0' usage_metadata={'input_tokens': 16, 'output_tokens': 103, 'total_tokens': 119, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [17]:
from langchain_core.load import load, loads

load_chain = load(
    loaded_chain, secrets_map={"OPENAI_API_KEY": os.environ["OPENAI_API_KEY"]}
)

load_chain.invoke({"fruit": "사과"})

AIMessage(content='사과의 색상은 다양합니다. 일반적으로 빨간색, 초록색, 노란색 등 여러 가지 색상이 있으며, 품종에 따라 다르게 나타납니다. 예를 들어, 레드 딜리셔스는 주로 빨간색이고, 그라니 스미스는 초록색이며, 골든 딜리셔스는 노란색입니다. 또한, 일부 사과는 두 가지 이상의 색상이 섞여 있는 경우도 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 103, 'prompt_tokens': 16, 'total_tokens': 119, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_86d0290411', 'id': 'chatcmpl-BFvpocDAqZ4V7F3ZkYo477w8HqQgR', 'finish_reason': 'stop', 'logprobs': None}, id='run-f13ad5f6-4ffa-4da3-a405-aa2048fe478e-0', usage_metadata={'input_tokens': 16, 'output_tokens': 103, 'total_tokens': 119, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [18]:
with open("fruit_chain.json", "r") as fp:
    loaded_from_json_chain = json.load(fp)
    loads_chain = load(loaded_from_json_chain)

In [19]:
loads_chain.invoke({"fruit": "사과"})

AIMessage(content='사과의 색상은 다양합니다. 일반적으로 빨간색, 초록색, 노란색 등이 있으며, 일부 사과는 이들 색상이 혼합된 형태로도 나타납니다. 예를 들어, 레드 딜리셔스 사과는 진한 빨간색을 띠고, 그라니 스미스 사과는 선명한 초록색을 가지고 있습니다. 또한, 골든 딜리셔스 사과는 노란색을 띠고 있습니다. 사과의 색상은 품종에 따라 다르며, 익는 정도에 따라서도 변화할 수 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 133, 'prompt_tokens': 16, 'total_tokens': 149, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_86d0290411', 'id': 'chatcmpl-BFvqoqk2it3iUiSwmJnumwpJS6t9E', 'finish_reason': 'stop', 'logprobs': None}, id='run-0f17bf92-f16e-4e97-b1d5-30edbdc34631-0', usage_metadata={'input_tokens': 16, 'output_tokens': 133, 'total_tokens': 149, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})